In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import cv2
import matplotlib.pyplot as plt  # Import matplotlib.pyplot
from matplotlib import pyplot as plt
import numpy as np
import pytesseract
from PIL import Image
import sympy as sp
   

## Data Preprocessing

In [ ]:
image_size = (28, 28)
batch_size = 32
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    '/kaggle/input/handwritten-math-symbols/dataset',
    target_size=image_size,
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=True,
    seed=0,
    save_to_dir=None,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/kaggle/input/handwritten-math-symbols/dataset',
    target_size=image_size,
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=True,
    seed=0,
    save_to_dir=None,
    class_mode='categorical',
    subset='validation'
)


## Model Definition

In [ ]:
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (28, 28, 1)
num_classes = len(train_generator.class_indices)
cnn_model = create_cnn_model(input_shape, num_classes)
cnn_model.summary()



## Model Training

In [ ]:
history=cnn_model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator
)


## Create test data generator

In [ ]:
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/handwritten-math-symbols/dataset',
    target_size=(28, 28),
    batch_size=32,
    shuffle=True,
    seed=0,
    save_to_dir=None,
    color_mode='grayscale',
    class_mode='categorical',
)

# Evaluate the model on the test data
test_loss, test_accuracy = cnn_model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.4f}')


## Plot training & validation accuracy values

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()


## Prediction

In [ ]:
def preprocess_image(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (28, 28))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    return img

def predict_label(model, image):
    img = preprocess_image(image)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction, axis=1)[0]
    
    class_labels = {v: k for k, v in train_generator.class_indices.items()}  # Assuming train_generator is available
    return class_labels[predicted_class]

In [ ]:
from IPython.display import Image
# Example usage
image_path = '/kaggle/input/handwritten-math-symbols/dataset/8/1913.jpg'
Image(filename=image_path)



In [ ]:
predicted_label = predict_label(cnn_model, cv2.imread('/kaggle/input/handwritten-math-symbols/dataset/8/1913.jpg'))
print(f"The predicted label for the image is: {predicted_label}")

##  Segmentation Symbols and Solve

In [ ]:
def parse_and_solve_equation(equation_text):
    
    lhs, rhs = equation_text.split('=')
    lhs_expr = sp.sympify(lhs)
    rhs_expr = sp.sympify(rhs)
    equation = sp.Eq(lhs_expr, rhs_expr)
    solution = sp.solve(equation)
    return solution

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_image(image):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (28, 28))
    image = image / 255.0
    image = np.expand_dims(image, axis=(0, -1))
    return image

def predict_label(model, image, labels):
    img = preprocess_image(image)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction, axis=1)[0]
    return labels[predicted_class]

image = cv2.imread('/kaggle/input/mathematical-problem1/ss.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

labels = list(train_generator.class_indices.keys())

symbols = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)

    if w * h < 50:
        continue

    pad = 5
    x1, y1 = max(0, x - pad), max(0, y - pad)
    x2, y2 = min(gray.shape[1], x + w + pad), min(gray.shape[0], y + h + pad)
    roi = gray[y1:y2, x1:x2]

    predicted_label = predict_label(cnn_model, roi, labels)

  
   

    symbols.append((x, predicted_label, (x1, y1, x2, y2)))

symbols.sort(key=lambda x: x[0])

st = ""
for symbol in symbols:
    x, label, (x1, y1, x2, y2) = symbol
    if label == "add":
        st += "+"
    elif label == "div":
        st += "/"
    elif label == "dec":
        st += "."
    elif label == "mul":
        st += "*"
    elif label == "eq":
        st += "="
    elif label == "sub":
        st += "-"
    else:
        st += label

    print("Predicted Label:", label)
    plt.imshow(cv2.cvtColor(image[y1:y2, x1:x2], cv2.COLOR_BGR2RGB))
    plt.title('Predicted Label: ' + label)
    plt.axis('off')
    plt.show()

if 'x' in st or 'y' in st or 'z' in st:
    print(parse_and_solve_equation(st))
else:
    if '=' in st:
        st = st[:st.find('=')]
    print("Answer:", eval(st))
